In [1]:
# Import libraries.
import pandas as pd
import glob
import math
from sklearn.metrics import ndcg_score

In [2]:
# Initialize paths for files containing product-level ratings from 1-5, 5 being the most relevant.
#u1_rating_file_path="C:\\Users\\varsh\\Desktop\\UMich MADS\\16. June 2023\\Ratings_U1\\*.xlsx"
#u2_rating_file_path="C:\\Users\\varsh\\Desktop\\UMich MADS\\16. June 2023\\Ratings_U2\\*.xlsx"
#u3_rating_file_path="C:\\Users\\varsh\\Desktop\\UMich MADS\\16. June 2023\\Ratings_U3.xlsx"

u1_rating_file_path='../resources/eval/Ratings_U1/*.xlsx'
u2_rating_file_path='../resources/eval/Ratings_U2/*.xlsx'
u3_rating_file_path='../resources/eval/Ratings_U3.xlsx'

In [3]:
u1_file_paths=glob.glob(u1_rating_file_path)
u2_file_paths=glob.glob(u2_rating_file_path)
file_paths=[u1_file_paths, u2_file_paths]


In [4]:
path = '../resources/eval/Ratings_U2/Q1_Method1_Average.xlsx'

#path = '../resources/eval/Ratings_U2/Q1_Method2_Discount_Reward_Only.xlsx'

In [5]:
df=pd.read_excel(path)
df.dropna(subset=["Query ID", "User ID"], inplace=True)
# Ensure that each line item has product-level rating.
df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
# Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
df.dropna(subset=["Detailed Rating (product level)"], inplace=True)

In [6]:
df["ideal_rating"]=sorted(list(df["Detailed Rating (product level)"]), reverse=True)

In [7]:
df.head(3)

,product_id,review_id,star_rating,distance_original,distance_opposite,distance_adjusted,similarity_score,product_similarity_score,product_title,review_body,Query ID,Query Body,Ranking Method,User ID,Binary Rating,Detailed Rating (review level),Detailed Rating (product level),Relevance Indicator,ideal_rating
0,B00CAWXSC8,R2O32EMSO7YR4,5.0,0.111228,0.290072,0.111228,0.184277,0.184277,12 Pairs Mens 100% Cotton Black Non Elastic To...,These socks are what I've been looking for- co...,Q1,"Long thin cotton socks for men, need to be bre...",Average,U2,1.0,4.0,4.0,NaN,4.0
1,B00UU6IVXW,R3JOXZELATQ06U,3.0,0.112565,0.261713,0.112565,0.174472,0.174472,Hanes Men`s Cushion Crew Socks,"Not as thick as I expected, but not bad for su...",Q1,"Long thin cotton socks for men, need to be bre...",Average,U2,1.0,2.0,2.0,NaN,4.0
2,B00CU3F4O2,RA9NHHU7UTBOU,3.0,0.118856,0.294298,0.118856,0.128339,0.128339,Mens 100% Cotton Plain Work/Casual Socks (Pack...,We bought these for a family member who has sw...,Q1,"Long thin cotton socks for men, need to be bre...",Average,U2,1.0,3.0,3.0,NaN,3.0


In [8]:
df['Detailed Rating (product level)'].values.reshape(1,-1)

array([[4., 2., 3., 2., 4., 2., 2., 2., 2., 2.]])

In [9]:
df['ideal_rating'].values.reshape(1,-1)

array([[4., 4., 3., 2., 2., 2., 2., 2., 2., 2.]])

In [10]:
ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=10)

0.9267173621354229

In [11]:
# Function to calculate Normalized Discounted Cumulative Gain for each query, each ranking method, and each user.
def NDCG(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path):
    
    # Create file lists for Users 1 and 2 since they have similar rating file formats.
    u1_file_paths=glob.glob(u1_rating_file_path)
    u2_file_paths=glob.glob(u2_rating_file_path)
    file_paths=[u1_file_paths, u2_file_paths]
    user_df_dict={}
    
    # Loop through each file in the folders for Users 1 and 2.
    for file_path in file_paths:
        ndcg_dict={}
        for path in file_path:
            
            # Read each rating file as a dataframe and perform basic sanity checks.
            df=pd.read_excel(path)
            df.dropna(subset=["Query ID", "User ID"], inplace=True)
            # Ensure that each line item has product-level rating.
            df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
            # Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
            df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
            df.dropna(subset=["Detailed Rating (product level)"], inplace=True)
            
            # Calculate the rank, denominator, the fraction for DCG, the ideal rank, and the fraction for IDCG.
            #df["rank"]=list(range(1, len(df)+1))
            #df["denom"]=df["rank"].apply(lambda x: math.log2(x+1))
            #df["dcg_frac"]=df["Detailed Rating (product level)"].div(df["denom"])
            df["ideal_rating"]=sorted(list(df["Detailed Rating (product level)"]), reverse=True)
            #df["idcg_frac"]=df["ideal_rating"].div(df["denom"])
            # Calculate the DCG, the IDCG, and the NDCG@10.
            #dcg=df["dcg_frac"].sum()
            #idcg=df["idcg_frac"].sum()
            #ndcg_10=round(dcg/idcg, 3)
            ndcg_10 = ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=10)
            # Calculate the DCG, the IDCG, and the NDCG@5.
            #df=df.head()
            #dcg=df["dcg_frac"].sum()
            #idcg=df["idcg_frac"].sum()
            ndcg_5 = ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=5)
            # Calculate the DCG, the IDCG, and the NDCG@3.
            #df=df.head(3)
            #dcg=df["dcg_frac"].sum()
            #idcg=df["idcg_frac"].sum()
            ndcg_3 = ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=3)
            
            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in ndcg_dict.keys():
                ndcg_dict["Query ID"]=[df["Query ID"].iloc[0]]
            else:
                ndcg_dict["Query ID"].append(df["Query ID"].iloc[0])
            if "Ranking Method" not in ndcg_dict.keys():
                ndcg_dict["Ranking Method"]=[df["Ranking Method"].iloc[0]]
            else:
                ndcg_dict["Ranking Method"].append(df["Ranking Method"].iloc[0])
            if "User ID" not in ndcg_dict.keys():
                ndcg_dict["User ID"]=[df["User ID"].iloc[0]]
            else:
                ndcg_dict["User ID"].append(df["User ID"].iloc[0])
            if "NDCG@10" not in ndcg_dict.keys():
                ndcg_dict["NDCG@10"]=[ndcg_10]
            else:
                ndcg_dict["NDCG@10"].append(ndcg_10)
            if "NDCG@5" not in ndcg_dict.keys():
                ndcg_dict["NDCG@5"]=[ndcg_5]
            else:
                ndcg_dict["NDCG@5"].append(ndcg_5)
            if "NDCG@3" not in ndcg_dict.keys():
                ndcg_dict["NDCG@3"]=[ndcg_3]
            else:
                ndcg_dict["NDCG@3"].append(ndcg_3)

        # Convert the dictionary to a dataframe and tidy it up.
        u_df=pd.DataFrame(ndcg_dict)
        u_df1=u_df.loc[u_df["Query ID"]!="Q10"].copy()
        u_df2=u_df.loc[u_df["Query ID"]=="Q10"].copy()
        u_df=pd.concat([u_df1, u_df2])
        u_df.reset_index(drop=True, inplace=True)
        # Add each user's result dataframe as a value in a dictionary.
        user_df_dict[df["User ID"].iloc[0]]=u_df
    
    # Create a list of query IDs.
    queries=list(u_df["Query ID"].unique())
    
    # Loop through each query ID for User 3, whose rating file differs from those of Users 1 and 2.
    ndcg_dict={}
    for query in queries:
        
        # Read each sheet in the rating file as a dataframe and perform basic sanity checks.
        df=pd.read_excel(u3_rating_file_path, sheet_name=query)
        df.dropna(subset=["product_id","review_id"], inplace=True)
        
        # Separate the dataframe based on ranking method for ease of calculation.
        dfa=df.loc[df["Ranking Method"]=="Average"].copy()
        dfd=df.loc[df["Ranking Method"]=="Discounted Reward Only"].copy()
        dfda=df.loc[df["Ranking Method"]=="Discounted Reward with Adjustment"].copy()

        # Create a list of the dataframes corresponding to each ranking method.
        df_list=[dfa, dfd, dfda]

        # Loop through each dataframe in the list corresponding to a different ranking method.
        for dfx in df_list:
            # Calculate the rank, denominator, the fraction for DCG, the ideal rank, and the fraction for IDCG.
            #dfx["rank"]=list(range(1, len(dfx)+1))
            #dfx["denom"]=dfx["rank"].apply(lambda x: round(math.log2(x+1), 3))
            #dfx["dcg_frac"]=round(dfx["Detailed Rating (product level)"].div(dfx["denom"]), 3)
            dfx["ideal_rating"]=sorted(list(dfx["Detailed Rating (product level)"]), reverse=True)
            #dfx["idcg_frac"]=round(dfx["ideal_rating"].div(dfx["denom"]), 3)
            # Calculate the DCG, the IDCG, and the NDCG@10.
            #dcg=dfx["dcg_frac"].sum()
            #idcg=dfx["idcg_frac"].sum()
            #ndcg_10=round(dcg/idcg, 3)
            ndcg_10 = ndcg_score(dfx['ideal_rating'].values.reshape(1,-1), dfx['Detailed Rating (product level)'].values.reshape(1,-1), k=10)
            # Calculate the DCG, the IDCG, and the NDCG@5.
            #dfx=dfx.head()
            #dcg=dfx["dcg_frac"].sum()
            #idcg=dfx["idcg_frac"].sum()
            #ndcg_5=round(dcg/idcg, 3)
            ndcg_5 = ndcg_score(dfx['ideal_rating'].values.reshape(1,-1), dfx['Detailed Rating (product level)'].values.reshape(1,-1), k=5)
            # Calculate the DCG, the IDCG, and the NDCG@3.
            #dfx=dfx.head(3)
            #dcg=dfx["dcg_frac"].sum()
            #idcg=dfx["idcg_frac"].sum()
            #ndcg_3=round(dcg/idcg, 3)
            ndcg_3 = ndcg_score(dfx['ideal_rating'].values.reshape(1,-1), dfx['Detailed Rating (product level)'].values.reshape(1,-1), k=3)

            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in ndcg_dict.keys():
                ndcg_dict["Query ID"]=[dfx["Query ID"].iloc[0]]
            else:
                ndcg_dict["Query ID"].append(dfx["Query ID"].iloc[0])
            if "Ranking Method" not in ndcg_dict.keys():
                ndcg_dict["Ranking Method"]=[dfx["Ranking Method"].iloc[0]]
            else:
                ndcg_dict["Ranking Method"].append(dfx["Ranking Method"].iloc[0])
            if "User ID" not in ndcg_dict.keys():
                ndcg_dict["User ID"]=[dfx["User ID"].iloc[0]]
            else:
                ndcg_dict["User ID"].append(dfx["User ID"].iloc[0])
            if "NDCG@10" not in ndcg_dict.keys():
                ndcg_dict["NDCG@10"]=[ndcg_10]
            else:
                ndcg_dict["NDCG@10"].append(ndcg_10)
            if "NDCG@5" not in ndcg_dict.keys():
                ndcg_dict["NDCG@5"]=[ndcg_5]
            else:
                ndcg_dict["NDCG@5"].append(ndcg_5)
            if "NDCG@3" not in ndcg_dict.keys():
                ndcg_dict["NDCG@3"]=[ndcg_3]
            else:
                ndcg_dict["NDCG@3"].append(ndcg_3)

    # Convert the dictionary to a dataframe and tidy it up.
    u_df=pd.DataFrame(ndcg_dict)
    # Add User 3's result dataframe as a value in the dictionary that contains the result dataframes for Users 1 and 2.
    user_df_dict[u_df["User ID"].iloc[0]]=u_df
    
    # Concatenate all three users' result dataframes into one big dataframe.
    user_df=pd.DataFrame()
    for df in user_df_dict.keys():
        user_df=pd.concat([user_df, user_df_dict[df]])

    # Return the dictionary containing each user's result dataframes and the big dataframe containing all 3 users' NDCG results.
    return user_df_dict, user_df

In [28]:
d, df=NDCG(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path)

In [29]:
d.keys()

dict_keys(['U1', 'U2', 'U3'])

In [33]:
d['U3']

,Query ID,Ranking Method,User ID,NDCG@10,NDCG@5,NDCG@3
0,Q1,Average,U3,0.937298,0.796398,0.836053
1,Q1,Discounted Reward Only,U3,0.948429,0.859280,0.867211
2,Q1,Discounted Reward with Adjustment,U3,0.955816,0.887718,0.875000
3,Q2,Average,U3,0.928447,0.844322,0.796736
4,Q2,Discounted Reward Only,U3,0.922784,0.823362,0.786351
5,Q2,Discounted Reward with Adjustment,U3,0.900253,0.784449,0.704007
6,Q3,Average,U3,0.939093,0.865445,0.839237
7,Q3,Discounted Reward Only,U3,0.984730,0.958333,0.958333
8,Q3,Discounted Reward with Adjustment,U3,0.940755,0.870157,0.842879
9,Q4,Average,U3,0.894512,0.718559,0.734421


In [15]:
df

,Query ID,Ranking Method,User ID,NDCG@10,NDCG@5,NDCG@3
0,Q1,Average,U1,0.917250,0.805861,0.750000
1,Q1,Discounted Reward Only,U1,0.959709,0.860806,0.916667
2,Q1,Discounted Reward with Adjustment,U1,0.929303,0.797663,0.812500
3,Q2,Average,U1,0.981928,0.950301,0.923511
4,Q2,Discounted Reward Only,U1,0.909765,0.760480,0.755193
...,...,...,...,...,...,...
25,Q9,Discounted Reward Only,U3,0.937164,0.858848,0.834138
26,Q9,Discounted Reward with Adjustment,U3,0.979918,0.934031,0.949008
27,Q10,Average,U3,0.992575,0.969815,1.000000
28,Q10,Discounted Reward Only,U3,0.913807,0.828767,0.796736


In [16]:
# Function to calculate Mean Reciprocal Rank for each ranking method and each user.
def MRR(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path):
    
    # Create file lists for Users 1 and 2 since they have similar rating file formats.
    u1_file_paths=glob.glob(u1_rating_file_path)
    u2_file_paths=glob.glob(u2_rating_file_path)
    file_paths=[u1_file_paths, u2_file_paths]
    user_df_dict={}
    
    # Loop through each file in the folders for Users 1 and 2.
    for file_path in file_paths:
        mrr_dict={}
        for path in file_path:
            
            # Read each rating file as a dataframe and perform basic sanity checks.
            df=pd.read_excel(path)
            df.dropna(subset=["Query ID", "User ID"], inplace=True)
            # Ensure that each line item has product-level rating.
            df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
            # Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
            df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
            df.dropna(subset=["Detailed Rating (product level)"], inplace=True)
            
            # Calculate the rank, the reciprocal rank, and the reciprocal rank of the most relevant product.
            df["rank"]=list(range(1, len(df)+1))
            df["reciprocal_rank"]=round(1/df["rank"], 3)
            rr=df.loc[df["Relevance Indicator"]=="Most relevant", "reciprocal_rank"].iloc[0]
            
            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in mrr_dict.keys():
                mrr_dict["Query ID"]=[df["Query ID"].iloc[0]]
            else:
                mrr_dict["Query ID"].append(df["Query ID"].iloc[0])
            if "Ranking Method" not in mrr_dict.keys():
                mrr_dict["Ranking Method"]=[df["Ranking Method"].iloc[0]]
            else:
                mrr_dict["Ranking Method"].append(df["Ranking Method"].iloc[0])
            if "User ID" not in mrr_dict.keys():
                mrr_dict["User ID"]=[df["User ID"].iloc[0]]
            else:
                mrr_dict["User ID"].append(df["User ID"].iloc[0])
            if "Reciprocal Rank" not in mrr_dict.keys():
                mrr_dict["Reciprocal Rank"]=[rr]
            else:
                mrr_dict["Reciprocal Rank"].append(rr)
        
        # Convert the dictionary to a dataframe and tidy it up.
        u_df=pd.DataFrame(mrr_dict)
        u_df1=u_df.loc[u_df["Query ID"]!="Q10"].copy()
        u_df2=u_df.loc[u_df["Query ID"]=="Q10"].copy()
        u_df=pd.concat([u_df1, u_df2])
        u_df.reset_index(drop=True, inplace=True)
        # Add each user's result dataframe as a value in a dictionary.
        user_df_dict[df["User ID"].iloc[0]]=u_df
        
    # Create a list of query IDs.
    queries=list(u_df["Query ID"].unique())
    
    # Loop through each query ID for User 3, whose rating file differs from those of Users 1 and 2.
    mrr_dict={}
    for query in queries:
        
        # Read each sheet in the rating file as a dataframe and perform basic sanity checks.
        df=pd.read_excel(u3_rating_file_path, sheet_name=query)
        df.dropna(subset=["product_id","review_id"], inplace=True)

        # Separate the dataframe based on ranking method for ease of calculation.
        dfa=df.loc[df["Ranking Method"]=="Average"].copy()
        dfd=df.loc[df["Ranking Method"]=="Discounted Reward Only"].copy()
        dfda=df.loc[df["Ranking Method"]=="Discounted Reward with Adjustment"].copy()

        # Create a list of the dataframes corresponding to each ranking method.
        df_list=[dfa, dfd, dfda]

        # Loop through each dataframe in the list corresponding to a different ranking method.
        for dfx in df_list:
            
            # Calculate the rank, the reciprocal rank, and the reciprocal rank of the most relevant product.           
            dfx["rank"]=list(range(1, len(dfx)+1))
            dfx["reciprocal_rank"]=round(1/dfx["rank"], 3)
            rr=dfx.loc[dfx["Relevance Indicator"]=="Most relevant", "reciprocal_rank"].iloc[0]
            
            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in mrr_dict.keys():
                mrr_dict["Query ID"]=[dfx["Query ID"].iloc[0]]
            else:
                mrr_dict["Query ID"].append(dfx["Query ID"].iloc[0])
            if "Ranking Method" not in mrr_dict.keys():
                mrr_dict["Ranking Method"]=[dfx["Ranking Method"].iloc[0]]
            else:
                mrr_dict["Ranking Method"].append(dfx["Ranking Method"].iloc[0])
            if "User ID" not in mrr_dict.keys():
                mrr_dict["User ID"]=[dfx["User ID"].iloc[0]]
            else:
                mrr_dict["User ID"].append(dfx["User ID"].iloc[0])
            if "Reciprocal Rank" not in mrr_dict.keys():
                mrr_dict["Reciprocal Rank"]=[rr]
            else:
                mrr_dict["Reciprocal Rank"].append(rr)
                
    # Convert the dictionary to a dataframe and tidy it up.
    u_df=pd.DataFrame(mrr_dict)
    # Add User 3's result dataframe as a value in the dictionary that contains the result dataframes for Users 1 and 2.
    user_df_dict[u_df["User ID"].iloc[0]]=u_df
    
    # Concatenate all three users' result dataframes into one big dataframe.
    user_df=pd.DataFrame()
    for df in user_df_dict.keys():
        user_df=pd.concat([user_df, user_df_dict[df]])
            
    # Return the dictionary containing each user's result dataframes and the big dataframe containing all 3 users' MRR results.
    return user_df_dict, user_df

In [17]:
d, df=MRR(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path)

In [18]:
d.keys()

dict_keys(['U1', 'U2', 'U3'])

In [19]:
d['U3']

,Query ID,Ranking Method,User ID,Reciprocal Rank
0,Q1,Average,U3,1.000
1,Q1,Discounted Reward Only,U3,0.500
2,Q1,Discounted Reward with Adjustment,U3,1.000
3,Q2,Average,U3,0.200
4,Q2,Discounted Reward Only,U3,0.333
5,Q2,Discounted Reward with Adjustment,U3,0.100
6,Q3,Average,U3,0.250
7,Q3,Discounted Reward Only,U3,0.200
8,Q3,Discounted Reward with Adjustment,U3,0.333
9,Q4,Average,U3,0.333


In [20]:
df

,Query ID,Ranking Method,User ID,Reciprocal Rank
0,Q1,Average,U1,0.200
1,Q1,Discounted Reward Only,U1,0.500
2,Q1,Discounted Reward with Adjustment,U1,1.000
3,Q2,Average,U1,1.000
4,Q2,Discounted Reward Only,U1,0.500
...,...,...,...,...
25,Q9,Discounted Reward Only,U3,0.333
26,Q9,Discounted Reward with Adjustment,U3,1.000
27,Q10,Average,U3,1.000
28,Q10,Discounted Reward Only,U3,1.000


In [21]:
d['U3'].groupby(["Ranking Method"]).agg({"Reciprocal Rank": "mean"})

,Reciprocal Rank
Ranking Method,
Average,0.5449
Discounted Reward Only,0.3963
Discounted Reward with Adjustment,0.4583
